## Loss Functions

딥러닝의 핵심은 주어진 네트워크의 weight bias를 loss 가 작아지는 쪽으로 바꾸는 데 있다. 따라서 손실함수야 말로 학습을 어떻게 할 지 정의해주는 부분이라고 생각 할 수 있다. 

pytorch / pytorch lightning 에서는 기본적으로 회귀에서 사용되는 MSE, 분류에서 사용되는 Cross Entropy 뿐 아니라 다양한 손실함수를 정의하고 있으며, 이는 torch.nn.functional 에 사전 정의되어 내장되어 있다. 

> torch.nn.functional
* mse_loss      : element-wise mean squared error.
* cross_entropy : cross entropy loss between input(logit) and target(prob.).
* binary_cross_entropy : Binary Cross Entropy between the target and input probabilities.
* binary_cross_entropy_with_logits :  Binary Cross Entropy between target and input logits.
* kl_div : Kullback-Leibler divergence Loss
* l1_loss : mean element-wise absolute value difference.
* smooth_l1_loss : uses a squared term if the absolute element-wise error falls below beta and an L1 term otherwise.
* nll_loss : negative log likelihood loss.
* poisson_nll_loss : Poisson negative log likelihood loss.
* gaussian_nll_loss : Gaussian negative log likelihood loss.


또한 Metric 들도 다양하게 사용할 수 있는데, Accuracy, AUCROC 등등 torchmetrics 를 통해 사용할 수 있다.
https://torchmetrics.readthedocs.io/en/stable/

대부분의 경우 내장됨 함수를 사용하게 되겠지만, 필요하다면 임의의 loss / metric 을 정의해서 쓸 수 도 있는데, 그 방법은 아래와 같다.

In [2]:
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
import torch.optim as optim

import pytorch_lightning as pl
from pytorch_lightning.accelerators import accelerator
from torchmetrics import functional as FM
from torchinfo import summary

from torchvision.datasets import MNIST
import torchvision.transforms as transforms
import torch.utils.data as data
from torch.utils.data import DataLoader
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
class Onehot(object) :
    def __call__(self, sample):
        sample = sample
        target = np.eye(10)[sample] # 10x10 대각행열을 만들어서 그 중에 n번째 row 를 반환 0 --> (1,0,0,0,0....0)
        return torch.FloatTensor(target)
    

In [4]:
a = Onehot()
a(5)

tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

In [5]:
y_transform = transforms.Compose([Onehot()])         # target one-hot encoding 
x_transform = transforms.Compose([transforms.ToTensor()])  # image transform 

In [6]:
train_dataset = MNIST('', transform=x_transform, target_transform=y_transform, train=True)
test_dataset = MNIST('', transform=x_transform, target_transform=y_transform, train=False)

In [7]:
batch_size = 128
trainDatLoader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valDataLoader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [8]:
class Model(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(28*28, 64)
        self.linear2 = nn.Linear(64, 32)
        self.linear3 = nn.Linear(32, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.flatten(x)
        x1 = self.linear1(x)
        x1 = self.relu(x1)
        x2 = self.linear2(x1)
        x2 = self.relu(x2)
        x3 = self.linear3(x2)
        return x3
               

In [9]:
def custom_loss_mse(pred, target):
    error = torch.mean(torch.square(pred-target))
    return error

In [10]:
def custom_mean_abs_error(y_pred, y_true):
    error = torch.abs(torch.mean(y_true - y_pred))
    return error 

In [11]:
class myModel(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.layers = Model()


    def forward(self, x):
        out = self.layers(x)
        out = torch.softmax(out, dim=-1) 
        return(out)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = custom_loss_mse(y_pred, y)
        error = custom_mean_abs_error(y_pred, y)
        metrics = {'loss' : loss, 'error' : error}
        self.log_dict(metrics)
        return loss


    def validation_step(self, batch, batch_idx):
        
    